In [1]:
# Dependencies
import pandas as pd
import os


In [2]:
# File Paths
temp_csv = os.path.join("Data","Land_Temp","land_temps.csv")
export_csv = os.path.join("Data", "Aquaculture", "Exports.csv")
import_csv = os.path.join("Data", "Aquaculture", "Imports.csv")
catfish_trout_csv = os.path.join("Data", "Aquaculture", "Catfish_Trout.csv")

In [28]:
# Load DataFrames
temp_df = pd.read_csv(temp_csv)
exports_df = pd.read_csv(export_csv)
imports_df = pd.read_csv(import_csv)
domestic_df = pd.read_csv(catfish_trout_csv)

## Clean Imports/Exports

In [4]:
# Concat Dataframes
commercial_df = pd.concat([imports_df, exports_df])
# Drop Source Col
commercial_df.drop(labels="SOURCE_ID", axis=1, inplace=True)
# Split Attribute Description
attr_split_df = commercial_df.ATTRIBUTE_DESC.str.split(pat=", ", expand=True)
commercial_df = pd.merge(left=commercial_df, right=attr_split_df, how="left", left_index=True, right_index=True)
commercial_df.rename(columns={0:"DIRECTION", 1:"MEASURE"}, inplace=True)

In [15]:
commercial_df

,HS_CODE,COMMODITY_DESC,GEOGRAPHY_CODE,GEOGRAPHY_DESC,ATTRIBUTE_DESC,UNIT_DESC,YEAR_ID,TIMEPERIOD_ID,AMOUNT,DIRECTION,MEASURE
0,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,"US Import, QTY",KG,1989,1,46682,US Import,QTY
0,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,"US Import, QTY",KG,1989,1,46682,US Export,QTY
0,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,"US Export, QTY",KG,1989,2,17183,US Import,QTY
0,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,"US Export, QTY",KG,1989,2,17183,US Export,QTY
1,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,"US Import, QTY",KG,1989,2,37354,US Import,QTY
...,...,...,...,...,...,...,...,...,...,...,...
548421,301110090.0,"ORNAMENTAL FRESHWATER FISH, LIVE",7970,Malawi,"US Import, VLU",U.S.$,2015,10,2145,US Import,VLU
548422,301110090.0,"ORNAMENTAL FRESHWATER FISH, LIVE",7970,Malawi,"US Import, VLU",U.S.$,2016,2,2603,US Import,VLU
548423,301110090.0,"ORNAMENTAL FRESHWATER FISH, LIVE",7970,Malawi,"US Import, VLU",U.S.$,2016,4,5541,US Import,VLU
548424,301190000.0,"ORNAMENTAL FISH (OTHER THAN FRESHWATER), LIVE",7970,Malawi,"US Import, VLU",U.S.$,2012,1,2926,US Import,VLU


## Clean Temperature Data
- Split Date into columns
- Aggregate by Month (to match commercial data)

In [29]:
#Split 'dt' column into Year, Month, Day
temp_dates = temp_df.dt.str.split(pat="-", expand=True)
temp_df = pd.merge(left=temp_df, right=temp_dates, how="left", left_index=True, right_index=True)
temp_df.rename(columns={0:"YEAR_ID", 1:"MONTH_ID", 2:"DAY_ID"}, inplace=True)
temp_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,YEAR_ID,MONTH_ID,DAY_ID
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W,1849,01,01
1,1849-02-01,27.434,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W,1849,02,01
2,1849-03-01,28.101,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W,1849,03,01
3,1849-04-01,26.140,1.387,Abidjan,Côte D'Ivoire,5.63N,3.23W,1849,04,01
4,1849-05-01,25.427,1.200,Abidjan,Côte D'Ivoire,5.63N,3.23W,1849,05,01


In [ ]:
# Aggregate by year/month/country
temp_gb = temp_df.groupby(by=["Country", "YEAR_ID", "MONTH_ID"])

## Align Country Names

In [6]:
# Let's make sets of the country names
countries_com = set(commercial_df.GEOGRAPHY_DESC)
countries_temp = set(temp_df.Country)
print(f"There are {len(countries_com)} country names in the commercial dataset")
print(f"There are {len(countries_temp)} country names in the temperature dataset")

There are 225 country names in the commercial dataset
There are 49 country names in the temperature dataset


In [7]:
#Lets see how many of the countries are in both datasets
countries_inter = countries_com.intersection(countries_temp)
print(f"There are {len(countries_inter)} country names that appear in both datasets")

There are 39 country names that appear in both datasets


In [8]:
# Since most of the temp countries fit inside of the commercial, let's see which ones do not
print("The following country names from the temperature data could not be found in the commercial data:")
print_statement = [print(each) for each in countries_temp.difference(countries_com)]

The following country names from the temperature data could not be found in the commercial data:
Burma
Syria
Germany
Congo (Democratic Republic Of The)
United States
Côte D'Ivoire
Sudan
Taiwan
South Africa
China


In [9]:
# We are going to rename the commercial_df to match the temp_df. We could go the other direction, but... 
# The temperature data labels make more sense
country_mapper = { 
    "Myanmar (Burma)" : "Burma",
    "China (Mainland)" : "China",
    "China (Taiwan)" : "Taiwan",
    "Congo (Kinshasa)" : "Congo (Democratic Republic Of The)",
    "German Democratic Republic" : "Germany",
    "Germany, Fed. Republic" : "Germany",
    "Ivory Coast" : "Côte D'Ivoire",
    "Republic of South Africa" : "South Africa",
}
commercial_df.GEOGRAPHY_DESC.replace(country_mapper, inplace=True)

Interestingly, there were no aquacultural exports to Sudan or Syria during this time

In [10]:
# And now the only countries that remain unmatched...
countries_com = set(commercial_df.GEOGRAPHY_DESC)
countries_temp = set(temp_df.Country)
print("These Countries could not be matched:")
print_statement = [print(each) for each in countries_temp.difference(countries_com)]

These Countries could not be matched:
Sudan
United States
Syria


There is just one issue remaining. We mapped two countries (The Germanies) onto one country name. We should also map their 'GEOGRAPHY_CODE' onto one ID as well. Let's see what the IDs are for Germany:

In [12]:
commercial_df[commercial_df.GEOGRAPHY_DESC == "Germany"]["GEOGRAPHY_CODE"].value_counts()

4280    19148
4290       16
Name: GEOGRAPHY_CODE, dtype: int64

It looks like there are only a few trades with '4290' code. This makes sense because the 'German Democratic Republic' was dissolved about a year into our commercial dataset. Let's check our intuitions...

In [17]:
commercial_df[commercial_df.GEOGRAPHY_CODE == 4290]

,HS_CODE,COMMODITY_DESC,GEOGRAPHY_CODE,GEOGRAPHY_DESC,ATTRIBUTE_DESC,UNIT_DESC,YEAR_ID,TIMEPERIOD_ID,AMOUNT,DIRECTION,MEASURE
240232,301100000.0,"FISH, ORNAMENTAL, LIVE",4290,Germany,"US Export, VLU",U.S.$,1989,1,2501,US Import,VLU
240232,301100000.0,"FISH, ORNAMENTAL, LIVE",4290,Germany,"US Export, VLU",U.S.$,1989,1,2501,US Export,VLU
240233,NaN,Ornamental fish,4290,Germany,"US Export, VLU",U.S.$,1989,1,2501,US Import,VLU
240233,NaN,Ornamental fish,4290,Germany,"US Export, VLU",U.S.$,1989,1,2501,US Export,VLU
319090,306130020.0,"SHRIMPS AND PRAWNS, SHELL-ON, FROZEN",4290,Germany,"US Import, QTY",KG,1989,11,16000,US Import,QTY
319090,306130020.0,"SHRIMPS AND PRAWNS, SHELL-ON, FROZEN",4290,Germany,"US Import, QTY",KG,1989,11,16000,US Export,QTY
319091,306130020.0,"SHRIMPS AND PRAWNS, SHELL-ON, FROZEN",4290,Germany,"US Import, VLU",U.S.$,1989,11,122171,US Import,VLU
319091,306130020.0,"SHRIMPS AND PRAWNS, SHELL-ON, FROZEN",4290,Germany,"US Import, VLU",U.S.$,1989,11,122171,US Export,QTY
319092,NaN,"Shrimp, frozen",4290,Germany,"US Import, QTY",KG,1989,11,16000,US Import,QTY
319092,NaN,"Shrimp, frozen",4290,Germany,"US Import, QTY",KG,1989,11,16000,US Export,QTY


And there we go! Isn't history fun? Ok let's get back to the Data Cleaning. We are just going to map those few communist Germans into their new united Germany

In [21]:
commercial_df.GEOGRAPHY_CODE.replace({4290 : 4280} , inplace=True)

In [22]:
commercial_df[commercial_df.GEOGRAPHY_DESC == "Germany"]["GEOGRAPHY_CODE"].value_counts()

4280    19164
Name: GEOGRAPHY_CODE, dtype: int64

## Data Aggregation
- Groupby country, species